The point of this notebook is to do a quick prediction on some sample images with a pretrained network.

In [ ]:
import sys

In [ ]:
sys.path.append('../')
sys.path.insert(0, '../models/')

In [ ]:
import cPickle as pickle
import re
import glob
import os

import time

import theano
import theano.tensor as T
import numpy as np
import pandas as p
import lasagne as nn

In [ ]:
from utils import hms, architecture_string, get_img_ids_from_iter

In [ ]:
%pylab inline

rcParams['figure.figsize'] = 16, 6
# rcParams['text.color'] = 'red'
# rcParams['xtick.color'] = 'red'
# rcParams['ytick.color'] = 'red'

In [ ]:
np.set_printoptions(precision=3)
np.set_printoptions(suppress=True)

First we load the dump of the trained network.

In [ ]:
param_dump_path = '../dumps/2015_07_17_123003_PARAMSDUMP.pkl'

In [ ]:
param_model_data = pickle.load(open(param_dump_path, 'r'))

In [ ]:
param_model_data[-1].shape

In [ ]:
import basic_model as model

In [ ]:
l_out, l_ins = model.build_model()

In [ ]:
nn.layers.set_all_param_values(l_out, param_model_data)

In [ ]:
model_data_store = {}
model_data_store['chunk_size'] = model_data['chunk_size']
model_data_store['batch_size'] = model_data['batch_size']
model_data_store['data_loader_params'] = model_data['data_loader_params']
model_data_store['paired_transfos'] = model_data['paired_transfos']
print model_data.has_key('data_loader_no_transfos')
print model_data.has_key('data_loader_default_transfo_params')

In [ ]:
extra_param_dump_path = '../dumps/2015_07_17_123003_EXTRAS.pkl'

In [ ]:
pickle.dump(model_data_store, open(extra_param_dump_path, 'wb'))

In [ ]:
dump_path = '../dumps/2015_07_17_123003.pkl'

In [ ]:
model_data = pickle.load(open(dump_path, 'r'))

In [ ]:
model_data['batch_size']

In [ ]:
# Let's set the in and output layers to some local vars.
l_out = model_data['l_out']
l_ins = model_data['l_ins']

Some info about the architecture of the model:

In [ ]:
model_arch = architecture_string(model_data['l_out'])
# model_arch = ''

num_params = nn.layers.count_params(l_out)
model_arch += "\nNumber of parameters: %d.\n\n" % num_params

# Get some training/validation info.
selected_keys = ['acc_eval_train', 'acc_eval_valid',
                 'losses_eval_train', 'losses_eval_valid',
                 'metric_eval_train', 'metric_eval_valid',
                 'metric_cont_eval_train', 'metric_cont_eval_valid']
model_metrics = {key: model_data[key]
                 for key in selected_keys if key in model_data}

res_df = p.DataFrame(model_metrics)

model_arch += 'BEST/LAST KAPPA TRAIN: %.3f - %.3f.\n' % (
    res_df.metric_eval_train.max(),
    res_df.metric_eval_train.iloc[-1]
)
model_arch += 'BEST/LAST KAPPA VALID: %.3f - %.3f.\n' % (
    res_df.metric_eval_valid.max(),
    res_df.metric_eval_valid.iloc[-1]
)

model_arch += '\nBEST/LAST ACC TRAIN: %.2f - %.2f.\n' % (
    res_df.acc_eval_train.max() * 100,
    res_df.acc_eval_train.iloc[-1] * 100
)

model_arch += 'BEST/LAST ACC VALID: %.2f - %.2f.\n' % (
    res_df.acc_eval_valid.max() * 100,
    res_df.acc_eval_valid.iloc[-1] * 100
)

model_arch += '\nTOTAL TRAINING TIME: %s' % \
              hms(model_data['time_since_start'])

print model_arch

*Note:* very long training time (80 hours!) is because of the (slow) AWS GPU and something special in the generator process which I only added at the end (the extra width cropping). You can get similar performance in (less than) 1 day with a GTX 980.

*Extra note:* if you have read my [blog post](http://jeffreydf.github.io/diabetic-retinopathy-detection/) you might notice the accuracy being much higher here than at the model image at the end (around 84% vs 80%). This is not (really) because this model is better but because this model used a lower *y_pow* (namely, *y_pow=1*). *y_pow* specifies to which power to raise the predictions before calculating the loss (see losses.py) and higher *y_pow* (mostly *y_pow=2*) gives much lower accuracy (around 80%) but used to give higher kappa scores. In the end they seemed to give similar kappa scores.

Some more kappa specific metrics:

In [ ]:
train_conf_mat, hist_rater_a, \
        hist_rater_b, train_nom, \
        train_denom = model_data['metric_extra_eval_train'][-1]

In [ ]:
valid_conf_mat, hist_rater_a, \
        hist_rater_b, valid_nom, \
        valid_denom = model_data['metric_extra_eval_valid'][-1]

In [ ]:
# Normalised train confusion matrix (with argmax decoding).
print train_conf_mat / train_conf_mat.sum()

In [ ]:
# Normalised validation confusion matrix (with argmax decoding).
print valid_conf_mat / valid_conf_mat.sum()

Setting up some Theano / Lasagne things to get some predictions.

In [ ]:
chunk_size = model_data['chunk_size'] * 2
batch_size = model_data['batch_size']

print "Batch size: %i." % batch_size
print "Chunk size: %i." % chunk_size

In [ ]:
output = nn.layers.get_output(l_out, deterministic=True)
input_ndims = [len(nn.layers.get_output_shape(l_in))
               for l_in in l_ins]
xs_shared = [nn.utils.shared_empty(dim=ndim)
             for ndim in input_ndims]
idx = T.lscalar('idx')

givens = {}
for l_in, x_shared in zip(l_ins, xs_shared):
    givens[l_in.input_var] = x_shared[idx * batch_size:(idx + 1) * batch_size]

compute_output = theano.function(
    [idx],
    output,
    givens=givens,
    on_unused_input='ignore'
)

In [ ]:
# Do transformations per patient instead?
if 'paired_transfos' in model_data:
    paired_transfos = model_data['paired_transfos']
else:
    paired_transfos = False
    
print paired_transfos

In [ ]:
xs_shared

We're going to test on some train images, so loading the training set labels.

In [ ]:
train_labels = p.read_csv(os.path.join('../data/trainLabels.csv'))

In [ ]:
print train_labels.head(5)

In [ ]:
# Get all patient ids.
patient_ids = sorted(set(get_img_ids_from_iter(train_labels.image)))

In [ ]:
num_chunks = int(np.ceil((2 * len(patient_ids)) / float(chunk_size)))

In [ ]:
# Where all the images are located: 
# it looks for [img_dir]/[patient_id]_[left or right].jpeg
img_dir = '/home/ubuntu/digits-server/train/'

Using the DataLoader to set up the parameters, you could replace it with something much simpler.

In [ ]:
from generators import DataLoader

In [ ]:
data_loader = DataLoader()
new_dataloader_params = model_data['data_loader_params']
new_dataloader_params.update({'images_test': patient_ids})
new_dataloader_params.update({'labels_test': train_labels.level.values})
new_dataloader_params.update({'prefix_train': img_dir})
data_loader.set_params(new_dataloader_params)

The next function is going to iterate over a test generator to get the outputs.

In [ ]:
def do_pred(test_gen):
    outputs = []

    for e, (xs_chunk, chunk_shape, chunk_length) in enumerate(test_gen()):
        num_batches_chunk = int(np.ceil(chunk_length / float(batch_size)))

        print "Chunk %i/%i" % (e + 1, num_chunks)
        print chunk_shape, chunk_length

        print "  load data onto GPU"
        for x_shared, x_chunk in zip(xs_shared, xs_chunk):
            x_shared.set_value(x_chunk)

        print "  compute output in batches"
        outputs_chunk = []
        for b in xrange(num_batches_chunk):
            out = compute_output(b)
            outputs_chunk.append(out)

        outputs_chunk = np.vstack(outputs_chunk)
        outputs_chunk = outputs_chunk[:chunk_length]

        outputs.append(outputs_chunk)

    return np.vstack(outputs), xs_chunk

We get the default "no transformation" parameters for the model.

In [ ]:
no_transfo_params = model_data['data_loader_params']['no_transfo_params']

print no_transfo_params

And set up the test generator on the first 256 patients of the training set (512 images).

In [ ]:
# The default gen with "no transfos".
test_gen = lambda: data_loader.create_fixed_gen(
    data_loader.images_test[:128*2],
    chunk_size=chunk_size,
    prefix_train=img_dir,
    prefix_test=img_dir,
    transfo_params=no_transfo_params,
    paired_transfos=paired_transfos,
)

Then we can get some predictions.

In [ ]:
%%time
outputs_orig, chunk_orig = do_pred(test_gen)

Explore some of the predictions.

In [ ]:
from metrics import continuous_kappa

In [ ]:
outputs_labels = np.argmax(outputs_orig, axis=1)

kappa_eval = continuous_kappa(
                outputs_labels,
                train_labels.level.values[:outputs_labels.shape[0]],
            )

metric, conf_mat, \
    hist_rater_a, hist_rater_b, \
    nom, denom = kappa_eval
    
print 'Kappa %.4f' % metric, '\n'
print conf_mat, '\n'
print nom, '\n'
print nom / nom.sum(), nom.sum()

Bit high of a kappa but this is because: 

1. There is a gap between the train and validation kappa.
2. This is a small sample.

Let's discriminate between train / validation.

In [ ]:
train_imgs = set(data_loader.images_train_0)
valid_idx = [0  if img in train_imgs else 1 for img in data_loader.images_test]

In [ ]:
df_preds = p.DataFrame([train_labels.image[:outputs_labels.shape[0]],
                        outputs_labels,
                        train_labels.level.values[:outputs_labels.shape[0]],
                       np.repeat(valid_idx, 2)[:outputs_labels.shape[0]]]).T
df_preds.columns = ['image', 'pred', 'true', 'valid']

The misclassifications:

In [ ]:
df_preds[df_preds.pred != df_preds.true]

Let's look at some sample activations:

In [ ]:
diag_out = theano.function(
    [idx],
    nn.layers.get_output(nn.layers.get_all_layers(l_out), deterministic=True),
    givens=givens,
    on_unused_input="ignore"
)

In [ ]:
diag_result = np.asarray(diag_out(0))

In [ ]:
# The input images.
diag_result[0].shape

In [ ]:
def plot_rollaxis(im, figsize=(15, 15), 
                  zmuv_mean=data_loader.zmuv_mean, 
                  zmuv_std=data_loader.zmuv_std,
                 norm=True, ax=None):
    if not ax:
        fig, ax = plt.subplots(1, figsize=figsize)
        
    if norm:
        ax.imshow((zmuv_std[0] + 0.05) * np.rollaxis(im, 0, 3) + zmuv_mean[0])
    else:
        ax.imshow(np.rollaxis(im, 0, 3))
        
    return ax

In [ ]:
plot_rollaxis(diag_result[0][1])

Do keep in mind, we work in "chunks" and only the last "chunk" is still loaded on the GPU.

Since a chunk is 256 images, we can subset those predictions.

In [ ]:
df_chunk = df_preds[-128*2:]
df_chunk['idx'] = np.repeat(range(128), 2)

print df_chunk

In [ ]:
# To print some output for a layer. (Hacky / quick.)
def print_output(layer_out, norm=False):
    fig, ax = plt.subplots(1, 2, sharey=True, figsize=(15, 200))

    for i, elem in enumerate(np.asarray(layer_out)[:2]):
        print elem.shape
        
        if norm:
            ax[i].imshow(np.concatenate(elem, axis=0), cmap=plt.cm.gray, 
                         vmin=np.asarray(layer_out).min(),
                         vmax=np.asarray(layer_out).max())
        else:        
            ax[i].imshow(np.concatenate(elem, axis=0), cmap=plt.cm.gray)

So, if we take index 8, we should see:

In [ ]:
idx = 8

In [ ]:
df_chunk[df_chunk.idx == idx]

In [ ]:
plot_rollaxis(diag_result[0][2*idx+0])  # Left.
plot_rollaxis(diag_result[0][2*idx+1])  # Right.

Let's see the output of one filter of the first layer for the left eye of that patient:

In [ ]:
fig, ax = plt.subplots(figsize=(15, 15))

ax.imshow(diag_result[1][2*idx+0][24], cmap=plt.cm.gray)

If you do this for some images with microaneurysms, most of the time you will see them getting "detected".

You can then also follow this "detection" through the following layers.

To get the activations for both eyes for a certain layer, we can do:

In [ ]:
print_output(diag_result[4][2*idx:], norm=False)

Another one:

In [ ]:
idx = 13

In [ ]:
df_chunk[df_chunk.idx == idx]

In [ ]:
plot_rollaxis(diag_result[0][2*idx+0])  # Left.
plot_rollaxis(diag_result[0][2*idx+1])  # Right.

Notice **the camera artifacts**!

In [ ]:
fig, ax = plt.subplots(figsize=(15, 15))

ax.imshow(diag_result[1][2*idx+0][10], cmap=plt.cm.gray)

In [ ]:
print_output(diag_result[3][2*idx:], norm=False)